<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/ML_spark_classifica%C3%A7%C3%A3o_RegrLogistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 52.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=15e5d0f4a1a6e75e5822c0f93abd1c797dd4067ebd9587e7d641acaf1fc82b5a
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('Reg_Log').getOrCreate()

In [4]:
churn = spark.read.csv('Churn.csv', header=True, inferSchema=True, sep=';')
churn.show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       

In [5]:
from pyspark.ml.feature import RFormula

In [8]:
Rformula = RFormula(formula='Exited ~ .', featuresCol='independente', labelCol='dependente')
churn_rformula = Rformula.fit(churn).transform(churn)
churn_rformula.select('independente','dependente').show(truncate=False)

+----------------------------------------------------------------+----------+
|independente                                                    |dependente|
+----------------------------------------------------------------+----------+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]        |1.0       |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]  |0.0       |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]  |1.0       |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])          |0.0       |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0]   |0.0       |
|[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0       |
|[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0]           |0.0       |
|[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0       |
|[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0]   |0.0       |
|[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0] 

In [9]:
churn_treino, churn_teste = churn_rformula.randomSplit([0.8,0.2])

In [10]:
churn_treino.count(), churn_teste.count()

(7998, 2002)

In [11]:
from pyspark.ml.classification import LogisticRegression

In [12]:
logistic = LogisticRegression(featuresCol='independente',labelCol='dependente',maxIter=100,regParam=0.08)

In [13]:
modelo = logistic.fit(churn_treino)

In [14]:
resumo = modelo.summary

In [15]:
acuracia = resumo.accuracy
precisao = resumo.weightedPrecision
recall = resumo.weightedRecall
auc = resumo.areaUnderROC

In [17]:
print(acuracia, precisao, recall, auc)

0.8034508627156789 0.7781897053709115 0.8034508627156789 0.7663623181567132


In [21]:
from os import truncate
previsao = modelo.transform(churn_teste)
previsao.select('dependente','prediction','probability','rawPrediction').show(truncate=False)

+----------+----------+----------------------------------------+------------------------------------------+
|dependente|prediction|probability                             |rawPrediction                             |
+----------+----------+----------------------------------------+------------------------------------------+
|1.0       |0.0       |[0.7346809814636831,0.2653190185363169] |[1.0185034206849053,-1.0185034206849053]  |
|1.0       |0.0       |[0.6712873851377477,0.32871261486225234]|[0.7140134811017247,-0.7140134811017247]  |
|1.0       |0.0       |[0.8115880397219309,0.18841196027806906]|[1.4603620276206266,-1.4603620276206266]  |
|1.0       |0.0       |[0.7482188182688816,0.2517811817311184] |[1.0891350891524585,-1.0891350891524585]  |
|0.0       |0.0       |[0.8532096844770152,0.14679031552298483]|[1.7600001938504475,-1.7600001938504475]  |
|0.0       |0.0       |[0.926625882469543,0.07337411753045697] |[2.535978654090731,-2.535978654090731]    |
|0.0       |0.0       |[0.86

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
avaliar = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='dependente',metricName='areaUnderROC')

In [24]:
areaunderROC = avaliar.evaluate(previsao)

In [25]:
print(areaunderROC)

0.7712338958860032


uma acurácia de 77%